# Proyecto_Predios ETL

#### Presentado por: Carlos Hidalgo Escobar

Se conecta a la base de datos donde se encuentra la data almacenada en la tabla de la primera fase de extraccion y staging (base_proye). Para asi continuar con el proceso de transformacion y load (carga) de los datos en nuestra base de datos dimensional pensada para almacenar los datos ya pulidos con sus calculos y todo para abrir el analisis de aquellos.

 *Database Connection*

In [ ]:
import os
import yaml
import psycopg2 
from psycopg2 import sql
from sqlalchemy import create_engine, text
import pandas as pd
import json
import numpy as np
from decimal import Decimal

In [2]:
#Carga de la confguracion de la base de datos
"""def load_config(file_path="config.yaml"):
    with open(file_path, "r") as file:
        return yaml.safe_load(file)"""
    
def load_config(file_path=None):
    if file_path is None:
        file_path = os.path.join(os.getcwd(), "config.yaml")  # Ruta absoluta

    with open(file_path, "r") as file:
        return yaml.safe_load(file)

In [3]:
config = load_config()
db_config = config["database"]

# Carga de credenciales
db_user = db_config["user"]
db_password = db_config["password"]
db_host = db_config["host"]
db_port = db_config["port"]
db_name = db_config["name"]

# DB connection
conn = psycopg2.connect(
    dbname="postgres",
    user=db_user,
    password=db_password,
    host=db_host,
    port=db_port
)
conn.autocommit = True

In [4]:
#Visualizamos los datos de la base de datos en un dataframe
engine = create_engine(f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")

with engine.connect() as conn:
    proye_db_df = pd.read_sql("SELECT * FROM base_proye", conn)
    
proye_db_df

,id,objeto_numerico,tipopred,avalpred_vigant,usu_vigant,actividad_vigant,estrato_vigant,area_vigant,terreno_vigant,predial_vigant,...,manzana,tipo_predio,actualizacion,avalpred_vigact,usu_vigact,actividad_vigact,estrato_vigact,area_vigact,terreno_vigact,cartera_vigact
0,1,76001010009080038002,P.V.R.,91742000,6,15,None,100,100,100000,...,4,MEJORA,RURAL,96329000,6,15,None,100,100,Y
1,2,51000001010200020102,P.V.R.,22873000,6,15,None,100,100,16000,...,4,MEJORA,RURAL,24017000,6,15,None,100,100,N
2,3,51000001010400010104,P.V.R.,37948000,6,15,None,100,100,29000,...,4,MEJORA,RURAL,39845000,6,15,None,100,100,N
3,4,51000001010500000105,V.R.,859144000,7,16,None,100,100,318000,...,4,NPH,RURAL,902101000,7,16,None,100,100,N
4,5,51000001010700000107,P.V.R.,44176000,6,15,None,100,100,30000,...,4,NPH,RURAL,46385000,6,15,None,100,100,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99996,02010003007809080055,CONST.,10113000,1,1,6.0,100,100,122000,...,3,PH,URBANO,13248000,1,1,6.0,100,100,N
99996,99997,02010003007909080055,CONST.,10113000,1,1,6.0,100,100,121000,...,3,PH,URBANO,13248000,1,1,6.0,100,100,N
99997,99998,02010003008009080055,CONST.,10113000,1,1,6.0,100,100,111000,...,3,PH,URBANO,13248000,1,1,6.0,100,100,N
99998,99999,02010004007609020004,CONST.,10113000,1,1,6.0,100,100,65000,...,4,PH,URBANO,13248000,1,1,6.0,100,100,N


# Transformación
*Limpieza de datos*

Se borra todos los datos que contengan cualquier valor 'NULL' de las columnas indicadas para comenzar con el resto de la etapa de transformacion, ya que no se tendrian en cuenta por ejemplo los predios que no tengan objeto_numerico ya que es el indicativo unico que define a la persona que le pertenece le predio.

In [ ]:
#Borrar todos los registros nulos en la columnas indicadas
proye_db_df = proye_db_df.dropna(subset=["objeto_numerico", "usu_vigant", "actividad_vigant", "avalpred_vigant", "predial_vigant", "avalpred_vigact"])

#Borrar todos los registros repetidos en la columna indicada
proye_db_df = proye_db_df.drop_duplicates(subset=['objeto_numerico'], keep='first')

proye_db_df

,id,objeto_numerico,tipopred,avalpred_vigant,usu_vigant,actividad_vigant,estrato_vigant,area_vigant,terreno_vigant,predial_vigant,...,manzana,tipo_predio,actualizacion,avalpred_vigact,usu_vigact,actividad_vigact,estrato_vigact,area_vigact,terreno_vigact,cartera_vigact
0,1,76001010009080038002,P.V.R.,91742000,6,15,None,100,100,100000,...,4,MEJORA,RURAL,96329000,6,15,None,100,100,Y
1,2,51000001010200020102,P.V.R.,22873000,6,15,None,100,100,16000,...,4,MEJORA,RURAL,24017000,6,15,None,100,100,N
2,3,51000001010400010104,P.V.R.,37948000,6,15,None,100,100,29000,...,4,MEJORA,RURAL,39845000,6,15,None,100,100,N
3,4,51000001010500000105,V.R.,859144000,7,16,None,100,100,318000,...,4,NPH,RURAL,902101000,7,16,None,100,100,N
4,5,51000001010700000107,P.V.R.,44176000,6,15,None,100,100,30000,...,4,NPH,RURAL,46385000,6,15,None,100,100,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99996,02010003007809080055,CONST.,10113000,1,1,6.0,100,100,122000,...,3,PH,URBANO,13248000,1,1,6.0,100,100,N
99996,99997,02010003007909080055,CONST.,10113000,1,1,6.0,100,100,121000,...,3,PH,URBANO,13248000,1,1,6.0,100,100,N
99997,99998,02010003008009080055,CONST.,10113000,1,1,6.0,100,100,111000,...,3,PH,URBANO,13248000,1,1,6.0,100,100,N
99998,99999,02010004007609020004,CONST.,10113000,1,1,6.0,100,100,65000,...,4,PH,URBANO,13248000,1,1,6.0,100,100,N


In [6]:
# Obtener valores únicos de una columna
valores_unicos = proye_db_df['usu_vigant'].unique()
valores_unicos2 = proye_db_df['actividad_vigant'].unique()

print(valores_unicos)
print(valores_unicos2)

['6' '7' '1' '8' '9' '10' '2' '14' '16' '4' '3' '5']
['15' '16' '1' '17' '18' '19' '3' '40' '25' '2' '24' '23' '31' '43' '42'
 '9' '11' '4' '5' '10' '7' '6' '13']


*Estandarización y manejo inicial de la data*

Se requiere ejecutar una estandarizacion de los datos indicados por el organismo publico y facilitar el manejo de ellos.

In [7]:
df_transformed = proye_db_df.copy() #Copia del dataframe proye_db_df para comienzo a la siguiente etapa

df_transformed.rename(columns={'id': 'id_predio'}, inplace=True) #Renombramiento de la columna 'id' a 'id_predio' para reutilzarla

#Eliminar la columna 'tipo_predio' porque no tiene relevancia en los calculos y puede genera confusion con la columna 'tipopred'
df_transformed = df_transformed.drop(columns=['tipo_predio'])

#Eliminar las columnas porque no tiene relevancia en los calculos.
df_transformed = df_transformed.drop(columns=['area_vigant', 'terreno_vigant','area_vigact', 'terreno_vigact', 'manzana'])

#Se agrega un 0 al inicio de cada objeto_numerico por el estandar que lo define (20 digitos).
df_transformed['objeto_numerico'] = df_transformed['objeto_numerico'].apply(lambda x: '0' + x if len(x) == 19 else x)

#Se agrega un 0 al inicio de los usu y actividad definiendo un estandar.
df_transformed[['usu_vigant', 'actividad_vigant', 'usu_vigact', 'actividad_vigact']] = df_transformed[['usu_vigant', 'actividad_vigant', 'usu_vigact', 'actividad_vigact']].map(lambda x: '0' + x if len(x) == 1 else x)

#Se concatenan el uso y la actividad de la vigencia actual.
df_transformed['uso_actividad_vigact'] = df_transformed['usu_vigact'].str.cat(df_transformed['actividad_vigact'])

#Redondear las columnas de los avaluos al multiplo 1000.
df_transformed['avalpred_vigant'] = df_transformed['avalpred_vigant'].round(-3)
df_transformed['avalpred_vigact'] = df_transformed['avalpred_vigact'].round(-3)

#Convertir las columnas 'estrato_vigact' y 'estrato_vigant' a Int64 y luego remplazar los vacios por la letra 'N' convirtiendo nuevamente la columna a string
df_transformed['estrato_vigact'] = pd.to_numeric(df_transformed['estrato_vigact'], errors='coerce').astype('Int64')
df_transformed['estrato_vigact'] = df_transformed['estrato_vigact'].astype(str).replace("<NA>", "N")
df_transformed['estrato_vigant'] = pd.to_numeric(df_transformed['estrato_vigant'], errors='coerce').astype('Int64')
df_transformed['estrato_vigant'] = df_transformed['estrato_vigant'].astype(str).replace("<NA>", "N")

#Agregar columna residencial o no residencial
df_transformed['tipo_res'] = df_transformed.apply(lambda row: 'Residencial' if row['uso_actividad_vigact'] == "0101" else "No Residencial", axis=1)

df_transformed

,id_predio,objeto_numerico,tipopred,avalpred_vigant,usu_vigant,actividad_vigant,estrato_vigant,predial_vigant,comuna,barrio,actualizacion,avalpred_vigact,usu_vigact,actividad_vigact,estrato_vigact,cartera_vigact,uso_actividad_vigact,tipo_res
0,1,76001010009080038002,P.V.R.,91742000,06,15,N,100000,B51,0,RURAL,96329000,06,15,N,Y,0615,No Residencial
1,2,51000001010200020102,P.V.R.,22873000,06,15,N,16000,B51,0,RURAL,24017000,06,15,N,N,0615,No Residencial
2,3,51000001010400010104,P.V.R.,37948000,06,15,N,29000,B51,0,RURAL,39845000,06,15,N,N,0615,No Residencial
3,4,51000001010500000105,V.R.,859144000,07,16,N,318000,B51,0,RURAL,902101000,07,16,N,N,0716,No Residencial
4,5,51000001010700000107,P.V.R.,44176000,06,15,N,30000,B51,0,RURAL,46385000,06,15,N,Y,0615,No Residencial
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99996,02010003007809080055,CONST.,10113000,01,01,6,122000,B02,1,URBANO,13248000,01,01,6,N,0101,Residencial
99996,99997,02010003007909080055,CONST.,10113000,01,01,6,121000,B02,1,URBANO,13248000,01,01,6,N,0101,Residencial
99997,99998,02010003008009080055,CONST.,10113000,01,01,6,111000,B02,1,URBANO,13248000,01,01,6,N,0101,Residencial
99998,99999,02010004007609020004,CONST.,10113000,01,01,6,65000,B02,1,URBANO,13248000,01,01,6,N,0101,Residencial


*Merge datos tarifa_predios*

Se realiza la carga de los datos del archivo .json tarifa_predios para unificar con el dataframe

In [ ]:
# Cargar datos desde el JSON
with open("./data/tarifas_predios.json", "r") as file:
    #tarifas_data = json.load(file)["tarifas_residencial_urbano_rural"
    tarifas_data = json.load(file, parse_float=Decimal)["tarifas_residencial_urbano_rural"]  # Mantiene precisión

# Convertir a DataFrame sin modificar num_tarifa para visualizar posteriormente
df_tarifas = pd.DataFrame(tarifas_data)

# Convertir listas a strings para mejor lectura
df_tarifas["uso_actividad_vigact"] = df_tarifas["uso_actividad_vigact"].apply(lambda x: ", ".join(x) if isinstance(x, list) else x)
df_tarifas["estrato_vigact"] = df_tarifas["estrato_vigact"].apply(lambda x: [str(e) for e in x] if isinstance(x, list) else [str(x)] if x else [])

print(df_tarifas.dtypes)  
df_tarifas

id_tarifa               object
num_tarifa              object
uso_actividad_vigact    object
estrato_vigact          object
dtype: object


,id_tarifa,num_tarifa,uso_actividad_vigact,estrato_vigact
0,T1,0.004,"0101, 0615, 1833, 1834","[1, N]"
1,T2,0.008,"0101, 1935, 2035","[2, 3, N]"
2,T3,0.011,"0101, 2137","[4, N]"
3,T4,0.013,"0101, 2238","[5, N]"
4,T5,0.014,"0101, 2339, 0918, 1019, 1120, 0940","[6, N]"
5,T6,0.01,"0101, 0409, 0410, 0411, 0442, 0817, 1423, 1424...","[3, N]"
6,T7,0.0145,"0202, 0203, 0204, 0306, 0307, 0308, 0512, 0513...",[N]
7,T8,0.010,"1426, 1425",[N]
8,T9,0.016,"0205, 1528, 1732, 1221",[N]
9,T10,0.033,1631,[N]


In [9]:
#Se realiza la asignacion de las tarifas mediante el match de los datos entre las columnas "uso_actividad_vigact" y "estrato_vigact"

# Función para asignar la tarifa
def asignar_tarifa(row):
    for _, tarifa in df_tarifas.iterrows():
        if row["uso_actividad_vigact"] in tarifa["uso_actividad_vigact"]:
            if row["estrato_vigact"] in tarifa["estrato_vigact"]:
                return pd.Series([tarifa["id_tarifa"], tarifa["num_tarifa"]])  # Corregido
    return pd.Series([None, None])  # Retorna None si no hay coincidencias

# Aplicar la función al DataFrame
df_transformed[["id_tarifa", "num_tarifa"]] = df_transformed.apply(asignar_tarifa, axis=1)

df_merged = df_transformed.copy()

#Renombramiento de la columna 'num_tarifa' a 'tarifa_vigact'
df_merged.rename(columns={'num_tarifa': 'tarifa_vigact'}, inplace=True)

df_merged

,id_predio,objeto_numerico,tipopred,avalpred_vigant,usu_vigant,actividad_vigant,estrato_vigant,predial_vigant,comuna,barrio,actualizacion,avalpred_vigact,usu_vigact,actividad_vigact,estrato_vigact,cartera_vigact,uso_actividad_vigact,tipo_res,id_tarifa,tarifa_vigact
0,1,76001010009080038002,P.V.R.,91742000,06,15,N,100000,B51,0,RURAL,96329000,06,15,N,Y,0615,No Residencial,T1,0.004
1,2,51000001010200020102,P.V.R.,22873000,06,15,N,16000,B51,0,RURAL,24017000,06,15,N,N,0615,No Residencial,T1,0.004
2,3,51000001010400010104,P.V.R.,37948000,06,15,N,29000,B51,0,RURAL,39845000,06,15,N,N,0615,No Residencial,T1,0.004
3,4,51000001010500000105,V.R.,859144000,07,16,N,318000,B51,0,RURAL,902101000,07,16,N,N,0716,No Residencial,T12,0.006
4,5,51000001010700000107,P.V.R.,44176000,06,15,N,30000,B51,0,RURAL,46385000,06,15,N,Y,0615,No Residencial,T1,0.004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99996,02010003007809080055,CONST.,10113000,01,01,6,122000,B02,1,URBANO,13248000,01,01,6,N,0101,Residencial,T5,0.014
99996,99997,02010003007909080055,CONST.,10113000,01,01,6,121000,B02,1,URBANO,13248000,01,01,6,N,0101,Residencial,T5,0.014
99997,99998,02010003008009080055,CONST.,10113000,01,01,6,111000,B02,1,URBANO,13248000,01,01,6,N,0101,Residencial,T5,0.014
99998,99999,02010004007609020004,CONST.,10113000,01,01,6,65000,B02,1,URBANO,13248000,01,01,6,N,0101,Residencial,T5,0.014


In [10]:
# Obtener valores únicos de una columna
valores_unicos = df_merged['tarifa_vigact'].unique()

conteo_nan = df_merged['tarifa_vigact'].isna().sum()
print(conteo_nan)

print(df_merged.dtypes)  
print(valores_unicos)

0
id_predio                int64
objeto_numerico         object
tipopred                object
avalpred_vigant          int64
usu_vigant              object
actividad_vigant        object
estrato_vigant          object
predial_vigant           int64
comuna                  object
barrio                  object
actualizacion           object
avalpred_vigact          int64
usu_vigact              object
actividad_vigact        object
estrato_vigact          object
cartera_vigact          object
uso_actividad_vigact    object
tipo_res                object
id_tarifa               object
tarifa_vigact           object
dtype: object
[Decimal('0.004') Decimal('0.006') Decimal('0.013') Decimal('0.008')
 Decimal('0.01') Decimal('0.014') Decimal('0.0145') Decimal('0.011')
 Decimal('0.033') Decimal('0.016')]


In [11]:
# Quitar la "T" y convertir a int
df_merged.loc[:, "id_tarifa"] = df_merged["id_tarifa"].str.replace("T", "", regex=False).astype(int)

df_merged

,id_predio,objeto_numerico,tipopred,avalpred_vigant,usu_vigant,actividad_vigant,estrato_vigant,predial_vigant,comuna,barrio,actualizacion,avalpred_vigact,usu_vigact,actividad_vigact,estrato_vigact,cartera_vigact,uso_actividad_vigact,tipo_res,id_tarifa,tarifa_vigact
0,1,76001010009080038002,P.V.R.,91742000,06,15,N,100000,B51,0,RURAL,96329000,06,15,N,Y,0615,No Residencial,1,0.004
1,2,51000001010200020102,P.V.R.,22873000,06,15,N,16000,B51,0,RURAL,24017000,06,15,N,N,0615,No Residencial,1,0.004
2,3,51000001010400010104,P.V.R.,37948000,06,15,N,29000,B51,0,RURAL,39845000,06,15,N,N,0615,No Residencial,1,0.004
3,4,51000001010500000105,V.R.,859144000,07,16,N,318000,B51,0,RURAL,902101000,07,16,N,N,0716,No Residencial,12,0.006
4,5,51000001010700000107,P.V.R.,44176000,06,15,N,30000,B51,0,RURAL,46385000,06,15,N,Y,0615,No Residencial,1,0.004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99996,02010003007809080055,CONST.,10113000,01,01,6,122000,B02,1,URBANO,13248000,01,01,6,N,0101,Residencial,5,0.014
99996,99997,02010003007909080055,CONST.,10113000,01,01,6,121000,B02,1,URBANO,13248000,01,01,6,N,0101,Residencial,5,0.014
99997,99998,02010003008009080055,CONST.,10113000,01,01,6,111000,B02,1,URBANO,13248000,01,01,6,N,0101,Residencial,5,0.014
99998,99999,02010004007609020004,CONST.,10113000,01,01,6,65000,B02,1,URBANO,13248000,01,01,6,N,0101,Residencial,5,0.014


*Calculos limites,descuento y valor a pagar predios*

Ya entrando en la parte mas importante de la proyeccion de los predios, el organismo publico consta de un estatuto tributario que contiene todos los acuerdos(leyes) que rigen como se deben realizar los calculos de todas las variables que participan en la proyeccion para asi definir un valor a pagar por cada predio.

In [ ]:
# Convertir a Decimal para mantener la diferencia en el formato
df_merged["tarifa_vigact"] = df_merged["tarifa_vigact"].apply(lambda x: Decimal(x))

SSMLV = 192172500 #Salario Mínimo Mensual Legal Vigente actual

#Funcion para asignar los indicadores segun las condiciones establecidas el Limite_3
def calcular_indicador(row):
    if ((row['estrato_vigact'] == 1) or (row['estrato_vigact'] == 2)) and (row['avalpred_vigact'] <= SSMLV):
        return 5.20
    elif ((row['estrato_vigact'] == 1) or (row['estrato_vigact'] == 2)) and (row['avalpred_vigact'] > SSMLV) and (row['cartera_vigact'] == 'N'):
        return 13.20
    elif (row['estrato_vigact'] != 1 and row['estrato_vigact'] != 2) and (row['avalpred_vigact'] > 0) and (row['cartera_vigact'] == 'N'):
        return 13.20
    elif row['cartera_vigact'] == 'Y':
        return 0.0
    elif row['cartera_vigact'] == 'N':
        return 13.20
    else:
        return 0.0

#Aplica la funcion calcular_indicador a las filas del df_transformed
df_merged['indicador_precio'] = df_merged.apply(calcular_indicador, axis=1)

#Calcular el Limite_1
df_merged["limite_1"] = df_merged.apply(
    lambda row: (Decimal(row["avalpred_vigact"]) * row["tarifa_vigact"]).quantize(Decimal('1')),
    axis=1
).astype("Int64")

#Calcular el Limite_2
df_merged["limite_2"] = (df_merged["predial_vigant"] * 2).round(10)
df_merged['limite_2'] = pd.to_numeric(df_merged['limite_2'], errors='coerce').astype('Int64')

#Calcular el Limite_3
df_merged['limite_3'] = df_merged.apply(lambda row: round(row['predial_vigant'] * (1 + row['indicador_precio'] / 100), -3) if row['indicador_precio'] > 0 else 0, axis=1)
df_merged['limite_3'] = pd.to_numeric(df_merged['limite_3'], errors='coerce').astype('Int64')

#Se toma el minimo entre los limites para escoger el impuesto_a_facturar
df_merged.loc[(df_merged['limite_1'] > 0) & (df_merged['indicador_precio'] > 0), 'impuesto_a_facturar'] = df_merged[['limite_1', 'limite_2', 'limite_3']].min(axis=1)
df_merged.loc[(df_merged['limite_1'] > 0) & (df_merged['indicador_precio'] == 0), 'impuesto_a_facturar'] = df_merged[['limite_1', 'limite_2']].min(axis=1)
df_merged.loc[df_merged['limite_1'] == 0, 'impuesto_a_facturar'] = 0

df_merged['favo_lim_vigact'] = np.select(
    [
        df_merged['impuesto_a_facturar'] == df_merged['limite_1'],
        df_merged['impuesto_a_facturar'] == df_merged['limite_2'],
        df_merged['impuesto_a_facturar'] == df_merged['limite_3']
    ],
    ['limite_1', 'limite_2', 'limite_3'],
    default='ninguno'
)

#Aplicar el descuento establecido a aplicar al impuesto_a_facturar
df_merged['descuento'] = df_merged['impuesto_a_facturar'] * 0.15
df_merged['descuento'] = pd.to_numeric(df_merged['descuento'], errors='coerce').astype('Int64')

#Calcular valor_a_pagar
df_merged['valor_a_pagar'] = df_merged['impuesto_a_facturar'] - df_merged['descuento']
df_merged['valor_a_pagar'] = pd.to_numeric(df_merged['valor_a_pagar'], errors='coerce').astype('Int64')

df_merged.head(20)

,id_predio,objeto_numerico,tipopred,avalpred_vigant,usu_vigant,actividad_vigant,estrato_vigant,predial_vigant,comuna,barrio,...,id_tarifa,tarifa_vigact,indicador_precio,limite_1,limite_2,limite_3,impuesto_a_facturar,favo_lim_vigact,descuento,valor_a_pagar
0,1,76001010009080038002,P.V.R.,91742000,06,15,N,100000,B51,0,...,1,0.004,0.0,385316,200000,0,200000,limite_2,30000,170000
1,2,51000001010200020102,P.V.R.,22873000,06,15,N,16000,B51,0,...,1,0.004,13.2,96068,32000,18000,18000,limite_3,2700,15300
2,3,51000001010400010104,P.V.R.,37948000,06,15,N,29000,B51,0,...,1,0.004,13.2,159380,58000,33000,33000,limite_3,4950,28050
3,4,51000001010500000105,V.R.,859144000,07,16,N,318000,B51,0,...,12,0.006,13.2,5412606,636000,360000,360000,limite_3,54000,306000
4,5,51000001010700000107,P.V.R.,44176000,06,15,N,30000,B51,0,...,1,0.004,0.0,185540,60000,0,60000,limite_2,9000,51000
5,6,13950029002100020021,CONST.,17761000,01,01,1,71000,B13,95,...,1,0.004,0.0,75308,142000,0,75308,limite_1,11296,64012
6,7,03020006001309020002,CONST.,297193000,01,01,5,3864000,B03,2,...,4,0.013,13.2,5061199,7728000,4374000,4374000,limite_3,656100,3717900
7,8,05970008000000000025,CONST.,72635000,01,01,3,726000,B05,97,...,2,0.008,13.2,615944,1452000,822000,615944,limite_1,92391,523553
8,9,63000003008800010088,V.R.,62301000,07,16,N,354000,B63,0,...,12,0.006,0.0,392496,708000,0,392496,limite_1,58874,333622
9,10,63000003008800020088,P.V.R.,29425000,06,15,N,5000,B63,0,...,1,0.004,0.0,123584,10000,0,10000,limite_2,1500,8500


In [13]:
# Obtener valores únicos de una columna
valores_unicos = df_merged['indicador_precio'].unique()

conteo_nan = df_merged['limite_1'].isna().sum()
print(conteo_nan)

print(df_merged.dtypes)  
print(valores_unicos)

0
id_predio                 int64
objeto_numerico          object
tipopred                 object
avalpred_vigant           int64
usu_vigant               object
actividad_vigant         object
estrato_vigant           object
predial_vigant            int64
comuna                   object
barrio                   object
actualizacion            object
avalpred_vigact           int64
usu_vigact               object
actividad_vigact         object
estrato_vigact           object
cartera_vigact           object
uso_actividad_vigact     object
tipo_res                 object
id_tarifa                object
tarifa_vigact            object
indicador_precio        float64
limite_1                  Int64
limite_2                  Int64
limite_3                  Int64
impuesto_a_facturar       Int64
favo_lim_vigact          object
descuento                 Int64
valor_a_pagar             Int64
dtype: object
[ 0.  13.2]


# Load

En la etapa de load (carga) de todos los datos ya transformados y listos para almacenarlos en nuestro data warehouse, se creara un modelo dimensional de los datos actuales tomando como tabla fact o principal sera la de los impuestos. Ya que contiene toda la informacion de los calculos y valores importantes que definen la proyeccion de los predios.

Nuestro modelo dimensional cuenta con tres tablas para reducir el tamaño del dataframe para lograr una mayor eficiencia en las solicitudes de los datos para su despliegue. 

Table names: dim_predio, dim_tarifa y fact_impuesto.

*DIM_PREDIO*

In [28]:
with engine.connect() as conn:
    conn.execute(text("""
        CREATE TABLE IF NOT EXISTS dim_predio (
            ID_PREDIO SERIAL PRIMARY KEY,
            OBJETO_NUMERICO VARCHAR(50),
            TIPOPRED VARCHAR(50),
            USU_VIGANT VARCHAR(50),
            ACTIVIDAD_VIGANT VARCHAR(50),
            ESTRATO_VIGANT VARCHAR(50),
            COMUNA VARCHAR(50),
            BARRIO VARCHAR(50),
            ACTUALIZACION VARCHAR(50),
            USU_VIGACT VARCHAR(50),
            ACTIVIDAD_VIGACT VARCHAR(50),
            ESTRATO_VIGACT VARCHAR(50),
            CARTERA_VIGACT VARCHAR(50),
            TIPO_RES VARCHAR(50)
        );
    """))
    conn.commit()  # Confirmar los cambios
    print("Tabla 'dim_predio' creada exitosamente en PostgreSQL.")

Tabla 'dim_predio' creada exitosamente en PostgreSQL.


*DIM_TARIFA*

In [29]:
with engine.connect() as conn:
    conn.execute(text("""
        CREATE TABLE IF NOT EXISTS dim_tarifa (
            ID_TARIFA SERIAL PRIMARY KEY,
            TARIFA_VIGACT float
        );
    """))
    conn.commit()  # Confirmar los cambios
    print("Tabla 'dim_tarifa' creada exitosamente en PostgreSQL.")

Tabla 'dim_tarifa' creada exitosamente en PostgreSQL.


*FACT_IMPUESTO*

In [30]:
with engine.connect() as conn:
    conn.execute(text("""
        CREATE TABLE IF NOT EXISTS fact_impuesto (
            ID_IMPUESTO SERIAL PRIMARY KEY,
            AVALPRED_VIGANT BIGINT,
            PREDIAL_VIGANT BIGINT,
            AVALPRED_VIGACT BIGINT,
            INDICADOR_PRECIO FLOAT,
            LIMITE_1 BIGINT,
            LIMITE_2 BIGINT,
            LIMITE_3 BIGINT,
            FAVO_LIM_VIGACT VARCHAR(50),
            IMPUESTO_A_FACTURAR BIGINT,
            DESCUENTO BIGINT,          
            VALOR_A_PAGAR BIGINT,
            ID_PREDIO INT REFERENCES dim_predio(ID_PREDIO) ON DELETE CASCADE,
            ID_TARIFA INT REFERENCES dim_tarifa(ID_TARIFA) ON DELETE SET NULL
        );
    """))
    conn.commit()  # Confirmar los cambios
    print("Tabla 'fact_impuesto' creada exitosamente en PostgreSQL.")

Tabla 'fact_impuesto' creada exitosamente en PostgreSQL.


### Carga de los datos

Despues de crear la base de datos dimensional, se toma el dataframe df_transformed y se divide en tres dataframes para proceder con la carga de los datos.

In [24]:
df_dim_tarifa = df_merged[['id_tarifa','tarifa_vigact']]
df_dim_predio = df_merged[['id_predio',
'objeto_numerico',
'tipopred',
'usu_vigant',
'actividad_vigant',
'estrato_vigant',
'comuna',
'barrio',
'actualizacion',
'usu_vigact',
'actividad_vigact',
'estrato_vigact',
'cartera_vigact',
'tipo_res']]
df_fact_impuesto = df_merged[['avalpred_vigant',
'predial_vigant',
'avalpred_vigact',
'indicador_precio',
'limite_1',
'limite_2',
'limite_3',
'impuesto_a_facturar',
'favo_lim_vigact',
'descuento',
'valor_a_pagar',
'id_predio',
'id_tarifa']]

In [25]:
df_dim_predio

,id_predio,objeto_numerico,tipopred,usu_vigant,actividad_vigant,estrato_vigant,comuna,barrio,actualizacion,usu_vigact,actividad_vigact,estrato_vigact,cartera_vigact,tipo_res
0,1,76001010009080038002,P.V.R.,06,15,N,B51,0,RURAL,06,15,N,Y,No Residencial
1,2,51000001010200020102,P.V.R.,06,15,N,B51,0,RURAL,06,15,N,N,No Residencial
2,3,51000001010400010104,P.V.R.,06,15,N,B51,0,RURAL,06,15,N,N,No Residencial
3,4,51000001010500000105,V.R.,07,16,N,B51,0,RURAL,07,16,N,N,No Residencial
4,5,51000001010700000107,P.V.R.,06,15,N,B51,0,RURAL,06,15,N,Y,No Residencial
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99996,02010003007809080055,CONST.,01,01,6,B02,1,URBANO,01,01,6,N,Residencial
99996,99997,02010003007909080055,CONST.,01,01,6,B02,1,URBANO,01,01,6,N,Residencial
99997,99998,02010003008009080055,CONST.,01,01,6,B02,1,URBANO,01,01,6,N,Residencial
99998,99999,02010004007609020004,CONST.,01,01,6,B02,1,URBANO,01,01,6,N,Residencial


Antes de realizar la insercion de los datos del dataframe df_dim_tarifa se hace unos breves ajustes del id_tarifa para evitar conflictos.

In [26]:
# Ordenar de forma ascendente
df_dim_tarifa = df_dim_tarifa.sort_values(by="id_tarifa", ascending=True).reset_index(drop=True)

# Eliminar duplicados manteniendo el primer valor encontrado
df_dim_tarifa = df_dim_tarifa.drop_duplicates(subset=['id_tarifa'], keep='first')

df_dim_tarifa

,id_tarifa,tarifa_vigact
0,1,0.004
12787,2,0.008
59769,3,0.011
65536,4,0.013
71833,5,0.014
80949,6,0.01
82006,7,0.0145
94917,9,0.016
94998,10,0.033
96398,12,0.006


In [27]:
df_fact_impuesto

,avalpred_vigant,predial_vigant,avalpred_vigact,indicador_precio,limite_1,limite_2,limite_3,impuesto_a_facturar,favo_lim_vigact,descuento,valor_a_pagar,id_predio,id_tarifa
0,91742000,100000,96329000,0.0,385316,200000,0,200000,limite_2,30000,170000,1,1
1,22873000,16000,24017000,13.2,96068,32000,18000,18000,limite_3,2700,15300,2,1
2,37948000,29000,39845000,13.2,159380,58000,33000,33000,limite_3,4950,28050,3,1
3,859144000,318000,902101000,13.2,5412606,636000,360000,360000,limite_3,54000,306000,4,12
4,44176000,30000,46385000,0.0,185540,60000,0,60000,limite_2,9000,51000,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,10113000,122000,13248000,13.2,185472,244000,138000,138000,limite_3,20700,117300,99996,5
99996,10113000,121000,13248000,13.2,185472,242000,137000,137000,limite_3,20550,116450,99997,5
99997,10113000,111000,13248000,13.2,185472,222000,126000,126000,limite_3,18900,107100,99998,5
99998,10113000,65000,13248000,13.2,185472,130000,74000,74000,limite_3,11100,62900,99999,5


*Insert datos df a tablas*

In [31]:
with engine.connect() as conn:
    df_dim_predio.to_sql("dim_predio", con=engine, if_exists="append", index=False)

print("Data guardada en la tabla 'dim_predio' exitosamente.")

Data guardada en la tabla 'dim_predio' exitosamente.


In [32]:
with engine.connect() as conn:
    df_dim_tarifa.to_sql("dim_tarifa", con=engine, if_exists="append", index=False)

print("Data guardada en la tabla 'dim_tarifa' exitosamente.")

Data guardada en la tabla 'dim_tarifa' exitosamente.


In [33]:
with engine.connect() as conn:
    df_fact_impuesto.to_sql("fact_impuesto", con=engine, if_exists="append", index=False)

print("Data guardada en la tabla 'fact_impuesto' exitosamente.")

Data guardada en la tabla 'fact_impuesto' exitosamente.
